In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.auto import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys
from pathlib import Path

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path
from src.utils.constants import Constants 
from src.datasets.eeg_dataset import EEGDataset
from src.models.gru import GRUModel

wandb.login()

2025-10-15 20:01:47,146 :: root :: INFO :: Initialising Utils
2025-10-15 20:01:47,147 :: root :: INFO :: Initialising Datasets
2025-10-15 20:01:47,171 :: root :: INFO :: Initialising Models
wandb: Currently logged in as: maikotrede (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
class CFG:
    seed = 42
    n_splits = 5
    data_path = '../../../data/'

    model_name = 'GRU'
    hidden_units = 128
    num_layers = 2
    target_size = 6 
    
    sampling_rate = 200 # Hz
    sequence_duration = 50 

    num_channels = 20 
    
    batch_size = 32
    num_workers = 0
    epochs = 30
    lr = 1e-3

CFG.sequence_length = CFG.sequence_duration * CFG.sampling_rate 

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(CFG.seed)

TARGETS = Constants.TARGETS


In [3]:
def get_dataloaders(df, fold_id):
    train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    train_dataset = EEGDataset(df=train_df, data_path=CFG.data_path, mode='train')

    valid_dataset = EEGDataset(df=valid_df, data_path=CFG.data_path, mode='valid')

    train_loader = DataLoader(
        train_dataset, batch_size=CFG.batch_size, shuffle=True,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=True
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.batch_size, shuffle=False,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=False
    )
    
    return train_loader, valid_loader

In [4]:


def run_training(df, DATA_PREPARATION_VOTE_METHOD):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    fold_scores = []

    for fold in range(CFG.n_splits):
        print(f"\n========== FOLD {fold} ==========")

        config = {
            "architecture": CFG.model_name, "hidden_units": CFG.hidden_units, "num_layers": CFG.num_layers,
            "fold": fold, "features": "raw_eeg", "sequence_duration": f"{CFG.sequence_duration}s",
            "optimizer": "AdamW", "learning_rate": CFG.lr, "batch_size": CFG.batch_size,
            "epochs": CFG.epochs, "seed": CFG.seed, "Scheduler": "CosineAnnealingLR" 
        }

        wandb.init(
            project="hms-aicomp-gru",
            name=f"gru-raw-eeg-fold{fold}", 
            tags=['gru-baseline', f'fold{fold}'],
            config=config
        )

        model = GRUModel(
            input_size=CFG.num_channels,
            hidden_size=CFG.hidden_units,
            num_layers=CFG.num_layers,
            num_classes=CFG.target_size
        )
        model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.epochs)
        loss_fn = nn.KLDivLoss(reduction='batchmean')
        train_loader, valid_loader = get_dataloaders(df, fold)

        best_val_loss = float('inf')
        best_model_path = None

        for epoch in range(CFG.epochs):
            print(f"   --- Epoch {epoch+1}/{CFG.epochs} ---")
            
            model.train()
            train_loss = 0
            for signals, labels in tqdm(train_loader, desc="Training"):
                signals, labels = signals.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(signals)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)
                loss.backward()
                # ==================================================================
                # %% GRADIENT MONITORING START
                # Optional: Log gradients to wandb to check for vanishing/exploding gradients
                total_norm = 0
                for name, p in model.named_parameters():
                    if p.grad is not None:
                        param_norm = p.grad.data.norm(2)
                        total_norm += param_norm.item() ** 2
                        
                        # Log the gradient norm for specific layers of interest
                        if "gru.weight_ih_l0" in name: # First layer input-hidden weights
                            wandb.log({"gradients/gru_layer_0_norm": param_norm.item()})
                        elif "fc.weight" in name: # Final fully-connected layer weights
                            wandb.log({"gradients/fc_layer_norm": param_norm.item()})

                total_norm = total_norm ** 0.5
                wandb.log({"gradients/total_norm": total_norm})
                # %% GRADIENT MONITORING END
                # ==================================================================
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                train_loss += loss.item() * signals.size(0)
                wandb.log({"train/loss": loss.item()})
            train_loss /= len(train_loader.dataset)

            model.eval()
            valid_loss = 0
            with torch.no_grad():
                for i, (signals, labels) in enumerate(tqdm(valid_loader, desc="Validation")):
                    signals, labels = signals.to(device), labels.to(device)
                    outputs = model(signals)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)
                    valid_loss += loss.item() * signals.size(0)


            valid_loss /= len(valid_loader.dataset)
            
            epoch_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")
            wandb.log({
                "epoch": epoch + 1, "train/epoch_loss": train_loss, "val/loss": valid_loss,
                "val/kl_div": valid_loss, "train/epoch_lr": epoch_lr
            })

            if valid_loss < best_val_loss:
                best_val_loss = valid_loss
                best_model_path = get_models_save_path() / "GRUModel" / DATA_PREPARATION_VOTE_METHOD / f'best_model_fold{fold}.pth'
                best_model_path.parent.mkdir(parents=True, exist_ok=True)
                torch.save(model.state_dict(), best_model_path)
                print(f"   New best model saved with validation loss: {best_val_loss:.4f}")
            scheduler.step()
        
        fold_scores.append(best_val_loss)
        wandb.summary['best_val_kl_div'] = best_val_loss
        if best_model_path:
            artifact = wandb.Artifact(f'model-fold{fold}', type='model')
            artifact.add_file(best_model_path)
            wandb.log_artifact(artifact)
            print(f"\nLogged artifact for fold {fold} with best validation loss: {best_val_loss:.4f}")
        else:
            print("\nNo best model was saved during training for this fold.")
        wandb.finish()
    return fold_scores

DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize"

print("Preparing data and creating folds...")
df = pd.read_csv(CFG.data_path + 'processed_data_sum_votes_window.csv') 

label_map = {t: i for i, t in enumerate(TARGETS)}
df['expert_consensus'] = df[TARGETS].idxmax(axis=1)

print('Train shape:', df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(df['fold'].value_counts())



Preparing data and creating folds...
Train shape: (17089, 12)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Folds created. Value counts per fold:
fold
0    3741
1    3703
2    3527
4    3081
3    3037
Name: count, dtype: int64


In [5]:
df

,eeg_id,spectrogram_id,min_offset,max_offset,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,fold
0,568657,789577333,0.0,16.0,20654,other_vote,0.0,0.000000,0.25,0.000000,0.166667,0.583333,0
1,582999,1552638400,0.0,38.0,20230,lpd_vote,0.0,0.857143,0.00,0.071429,0.000000,0.071429,0
2,642382,14960202,1008.0,1032.0,5955,other_vote,0.0,0.000000,0.00,0.000000,0.000000,1.000000,2
3,751790,618728447,908.0,908.0,38549,gpd_vote,0.0,0.000000,1.00,0.000000,0.000000,0.000000,3
4,778705,52296320,0.0,0.0,40955,other_vote,0.0,0.000000,0.00,0.000000,0.000000,1.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1188113564,0.0,0.0,16610,grda_vote,0.0,0.000000,0.00,0.000000,0.500000,0.500000,2
17085,4293843368,1549502620,0.0,0.0,15065,grda_vote,0.0,0.000000,0.00,0.000000,0.500000,0.500000,2
17086,4294455489,2105480289,0.0,0.0,56,other_vote,0.0,0.000000,0.00,0.000000,0.000000,1.000000,4
17087,4294858825,657299228,0.0,12.0,4312,other_vote,0.0,0.000000,0.00,0.000000,0.066667,0.933333,0


In [ ]:
if __name__ == '__main__':
    all_fold_scores = run_training(df, DATA_PREPARATION_VOTE_METHOD)
    cv_score = np.mean(all_fold_scores)
    print(f"\nCross-Validation Score (Mean KL Divergence across folds): {cv_score:.4f}")

Using device: cuda

========== FOLD 0 ==========


   --- Epoch 1/30 ---


Training:   0%|          | 0/417 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

   Epoch 1: Train Loss = 1.2492, Valid Loss = 1.3072, LR = 0.001000
   New best model saved with validation loss: 1.3072
   --- Epoch 2/30 ---


...

   Epoch 18: Train Loss = 0.5564, Valid Loss = 1.2544, LR = 0.000396
   --- Epoch 19/30 ---


Training:   0%|          | 0/417 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

   Epoch 19: Train Loss = 0.5214, Valid Loss = 1.2552, LR = 0.000345
   --- Epoch 20/30 ---


Training:   0%|          | 0/417 [00:00<?, ?it/s]